## Import Libraries

In [ ]:
import pandas as pd
from geonamescache import GeonamesCache
from unidecode import unidecode
import re
import numpy as np

## Import Data

In [2]:
data = pd.read_csv('Checkpoint01b.csv', index_col = [0])

## Get Countries in Headlines

I should have done this in Part 01, but I misunderstood!

In [3]:
headlines = [unidecode(line) for line in data['headline']]

In [4]:
country2 = np.full_like(np.arange(len(data)), np.nan, dtype = object)

In [5]:
data2 = data
data2['country2']=country2

In [6]:
gc = GeonamesCache()
countries = gc.get_countries()

In [7]:
for j in data2.index:
    for key in list(countries.keys()):
        name = countries[key]['name']
        match = re.search('\\b' + name + '\\b', data2.headline.loc[j])
        if match != None:
            data2.loc[j, 'country2'] = name    

In [8]:
data3 = data2.drop_duplicates(['headline', 'city']).reset_index().drop('index', axis = 1)

## Select a Single City Per Headline

I decided to use the most populous city, unless a country was in a headline.

In [9]:
city_data = pd.DataFrame(columns = ['index', 'key', 'city', 'country', 'population'])

In [10]:
for j in range(11):    
    city_info = pd.DataFrame(columns = ['key', 'city', 'country', 'population'])
    cities_list = gc.get_cities_by_name(data3.loc[j,'city'])
    if (not cities_list):
        city_info.loc[0]=np.nan 
        the_city = city_info.loc[0]
    elif data3.loc[j,'country2']==data3.loc[j,'country2']:
        the_country = gc.get_countries_by_names()[data3.loc[j,'country2']]
        iso = the_country['iso']
        i = 0
        for city in cities_list:
            if city[list(city.keys())[0]]['countrycode'] == iso:
                city_info.loc[i, 'key'] = list(city.keys())[0]
                city_info.loc[i,'city'] = city[list(city.keys())[0]]['name']
                city_info.loc[i,'country'] = city[list(city.keys())[0]]['countrycode']
                city_info.loc[i,'population'] = city[list(city.keys())[0]]['population']
            else:
                city_info.loc[0]=np.nan 
                the_city = city_info.loc[0]
            i+=1
        the_city = city_info.sort_values(by = ['population'], ascending = False).reset_index().loc[0]
    elif len(cities_list) > 1:
        i = 0
        for city in cities_list:
            city_info.loc[i, 'key'] = list(city.keys())[0]
            city_info.loc[i,'city'] = city[list(city.keys())[0]]['name']
            city_info.loc[i,'country'] = city[list(city.keys())[0]]['countrycode']
            city_info.loc[i,'population'] = city[list(city.keys())[0]]['population']
            i+=1
        the_city = city_info.sort_values(by = ['population'], ascending = False).reset_index().loc[0]
    else:
        i = 0
        city = cities_list[0]
        city_info.loc[i, 'key'] = list(city.keys())[0]
        city_info.loc[i,'city'] = city[list(city.keys())[0]]['name']
        city_info.loc[i,'country'] = city[list(city.keys())[0]]['countrycode']
        city_info.loc[i,'population'] = city[list(city.keys())[0]]['population']
        the_city = city_info.reset_index().loc[0]
    city_data = city_data.append(the_city, ignore_index = True)

In [11]:
city_data = city_data.drop('index', axis = 1)

## Get Longitude and Latitude

In [12]:
latitude = np.full_like(np.arange(len(city_data)), np.nan, dtype = float)
longitude = np.full_like(np.arange(len(city_data)), np.nan, dtype = float)

In [13]:
city_data['latitude'] = latitude
city_data['longitude'] = longitude

In [14]:
cities = gc.get_cities()

In [15]:
for i in range(len(city_data)):
    key = city_data.loc[i,'key']
    city_data.loc[i,'latitude'] = cities[key]['latitude']
    city_data.loc[i,'longitude'] = cities[key]['longitude']

In [16]:
city_data.head()

,key,city,country,population,latitude,longitude
0,292223,Dubai,AE,2956587,25.07725,55.30927
1,616052,Yerevan,AM,1093485,40.18111,44.51361
2,2240449,Luanda,AO,2776168,-8.83682,13.23432
3,3651438,Santa Elena,EC,42214,-2.22622,-80.85873
4,3911925,La Paz,BO,812799,-16.50000,-68.15000


In [22]:
print(sum(np.isnan(city_data.latitude)))
sum(np.isnan(city_data.longitude))

0


0

In [ ]:
city_data.to_csv('Checkpoint_02.csv')